In [1]:
import nest_asyncio
from crewai import LLM
from dotenv import load_dotenv
import os
import asyncio
from echo.constants import (
    DISCOVERY,
    DEMO,
    PRICING,
    NEGOTIATION
)


nest_asyncio.apply()

load_dotenv()

llm = LLM(
    model=os.getenv("FIREWORKS_MODEL_NAME"),
    base_url="https://api.fireworks.ai/inference/v1",
    api_key=os.getenv("FIREWORKS_API_KEY"),
)

In [2]:
from echo.runner import create_or_get_seller

inputs = {
    "seller": "Whatfix",
    "num_buyers": 10,
}

seller = asyncio.run(create_or_get_seller(inputs))

Creating seller for Whatfix
Adding Seller: Whatfix Data


In [3]:
from echo.runner import make_call


buyer_inputs = {
    **inputs,
    **seller,
    'n_competitors': 10,
}

clients = seller['seller_clients']['clients']

In [4]:
seller['seller_clients']['clients']

['Cardinal Health',
 'BMC Software',
 'Grant Thornton',
 'Coca Cola',
 'Equinix',
 'Sophos',
 'Sentry Insurance',
 'Marketboomer']

In [5]:
discovery_calls_data = asyncio.run(make_call(DISCOVERY, seller['seller_clients']['clients'], buyer_inputs))

Making discovery call for ['Cardinal Health', 'BMC Software', 'Grant Thornton', 'Coca Cola', 'Equinix', 'Sophos', 'Sentry Insurance', 'Marketboomer']
Getting analysis Data
Number of Clients:  8


Getting Data:   0%|          | 0/8 [00:00<?, ?it/s]

Getting Data for Cardinal Health
Adding Analysis Data to Vector Store


Getting Data:  12%|█▎        | 1/8 [00:00<00:05,  1.27it/s]

Getting Data for BMC Software
Data not found for client: BMC Software
Data not found for client: BMC Software
Adding Buyer: BMC Software Data


...

2025-02-12 21:15:59,256 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding discovery Transcript Data to Vector Store


2025-02-12 21:16:04,876 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding Analysis Data to Vector Store


Getting Data: 100%|██████████| 8/8 [04:06<00:00, 30.85s/it]


In [6]:
demo_calls_data = asyncio.run(make_call(DEMO, clients, buyer_inputs))

Making demo call for ['Cardinal Health', 'BMC Software', 'Grant Thornton', 'Coca Cola', 'Equinix', 'Sophos', 'Sentry Insurance', 'Marketboomer']
Getting analysis Data
Number of Clients:  8


Getting Data:   0%|          | 0/8 [00:00<?, ?it/s]2025-02-12 21:17:06,725 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Getting Data for Cardinal Health
Adding Research Data to Vector Store


2025-02-12 21:17:09,692 - 6258388992 - _common.py-_common:105 - INFO: Backing off send_request(...) for 0.0s (requests.exceptions.ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))
2025-02-12 21:17:09,905 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding demo Transcript Data to Vector Store


...

2025-02-12 21:20:18,629 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding demo Transcript Data to Vector Store


2025-02-12 21:20:25,467 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Retrying due to error: Failed to convert text into a Pydantic model due to the following error: Instructor does not support multiple tool calls, use List[Model] instead
Adding Analysis Data to Vector Store


Getting Data: 100%|██████████| 8/8 [03:47<00:00, 28.44s/it]


In [7]:
pricing_call_data = asyncio.run(make_call(PRICING, clients, buyer_inputs))

Making pricing call for ['Cardinal Health', 'BMC Software', 'Grant Thornton', 'Coca Cola', 'Equinix', 'Sophos', 'Sentry Insurance', 'Marketboomer']
Getting analysis Data
Number of Clients:  8


Getting Data:   0%|          | 0/8 [00:00<?, ?it/s]2025-02-12 21:20:54,188 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Getting Data for Cardinal Health
Data exists but pricing transcript not found
Retrying due to error: Failed to convert text into a Pydantic model due to the following error: Instructor does not support multiple tool calls, use List[Model] instead
Adding pricing Transcript Data to Vector Store


2025-02-12 21:21:26,566 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding Analysis Data to Vector Store


...

Getting Data:  88%|████████▊ | 7/8 [01:34<00:10, 10.63s/it]2025-02-12 21:22:29,010 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Getting Data for Marketboomer
Data exists but pricing transcript not found
Adding pricing Transcript Data to Vector Store


2025-02-12 21:22:34,124 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding Analysis Data to Vector Store


Getting Data: 100%|██████████| 8/8 [01:43<00:00, 12.99s/it]


In [8]:
negotiation_call_data = asyncio.run(make_call(NEGOTIATION, clients, buyer_inputs))

Making negotiation call for ['Cardinal Health', 'BMC Software', 'Grant Thornton', 'Coca Cola', 'Equinix', 'Sophos', 'Sentry Insurance', 'Marketboomer']
Getting analysis Data
Number of Clients:  8


Getting Data:   0%|          | 0/8 [00:00<?, ?it/s]2025-02-12 21:22:38,120 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Getting Data for Cardinal Health
Data exists but pricing transcript not found
Adding negotiation Transcript Data to Vector Store


2025-02-12 21:22:43,245 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding Analysis Data to Vector Store


...

Getting Data:  88%|████████▊ | 7/8 [01:02<00:09,  9.24s/it]2025-02-12 21:23:40,875 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Getting Data for Marketboomer
Data exists but pricing transcript not found
Adding negotiation Transcript Data to Vector Store


2025-02-12 21:23:46,443 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding Analysis Data to Vector Store


Getting Data: 100%|██████████| 8/8 [01:12<00:00,  9.05s/it]


## Queries

In [13]:
# len(index._storage_context.vector_store._collection.get()['metadatas'])

In [1]:
from echo.queries import get_queries

seller = "Whatfix"
queries = get_queries(seller)

In [2]:
print("Total queries: ", len(queries))

Total queries:  4


In [3]:
queries.keys()

dict_keys(['discovery', 'demo', 'pricing', 'negotiation'])

In [ ]:
# vector_index.storage_context.vector_store._collection.get()['metadatas']

In [4]:
from echo.query_executor import arun_queries
from echo.query_executor import ResponseFormat
from echo.query_executor import ContextExtractionMode
import nest_asyncio
import asyncio

nest_asyncio.apply()


inputs = {"buyer": "Sophos", "company_size": "Enterprise"}

responses = asyncio.run(
    arun_queries(
        queries,
        inputs=inputs,
        response_format=ResponseFormat.MARKDOWN,
        context_extraction_mode=ContextExtractionMode.QUERY_ENGINE,
    )
)

Running queries for call type discovery


Running Queries:   0%|          | 0/8 [00:00<?, ?it/s]

Running sub query What all pieces of information about the buyer are uncovered in discovery calls of successful deals?
Sub query context {'query': 'What all pieces of information about the buyer are uncovered in discovery calls of successful deals?', 'context': 'Relevant Context:\nThe pain points, objections, time lines, success indicators, budget constraints, competition, and decision committee members are all pieces of information about the buyer that are uncovered in discovery calls of successful deals.'}
Running sub query What all piece of information have we learnt about the buyer from discovery till now?
Sub query context {'query': 'What all piece of information have we learnt about the buyer from discovery till now?', 'context': 'Relevant Context:\nThe buyer, Sophos, is struggling with user onboarding and adoption, leading to low adoption rates and decreased productivity. They find their current training program time-consuming and costly, and they are challenged in measuring the

Running Queries:  12%|█▎        | 1/8 [00:07<00:50,  7.23s/it]

Trying to parse with Ast...
Running sub query What are the top priorities for the account to solve for?


...

Trying to parse with Ast...
Running sub query What are the closing tactics that have been successful in the past?


Overriding of current TracerProvider is not allowed


Sub query context {'query': 'What are the closing tactics that have been successful in the past?', 'context': "Relevant Context:\nThe successful closing tactics in the past have included offering a detailed quote, scheduling a follow-up call, assigning a customer success manager, providing a clear next step, involving multiple stakeholders in the call, sending a formal proposal, and acknowledging the buyer's requirements while emphasizing the benefits of the platform."}
Running final query What final closing tactics can be used for their account?


Running Queries: 100%|██████████| 4/4 [00:22<00:00,  5.74s/it]

Trying to parse with Ast...


In [5]:
len(responses.keys())

19

In [4]:
responses["Info To Cover"]

{'response': "# Call Preparation Document\n## Discovery Call with Shell\n### Overview\nThe goal of this discovery call is to uncover the needs and pain points of Shell, a potential buyer, and understand how Whatfix can help them address their challenges.\n\n### Information Uncovered so far\n#### Challenges and Pain Points\n* Maximize technology investments and driving user adoption\n* Balancing oil demand with climate goals\n* Slow user adoption of new tools, impacting productivity and efficiency\n* Integration with existing systems\n* High degree of customization to meet unique requirements\n* Concerns about cost and implementation timeline of a digital adoption platform\n\n#### Decision Committee Members\n* IT\n* Operations\n* Business Leaders\n* CEO Wael Sawan\n* Mark (Shell icon)\n* Shareholders\n\n### Missing Information to Uncover\n#### From Historical Data\n* Objections\n* Time lines\n* Success indicators\n* Budget constraints\n* Competition\n\n#### From Current Call\n* None, as

In [1]:
# from echo.query_executor import aget_query_response


# response, sub_queries_context = await aget_query_response(
#     query,
#     inputs,
#     response_format=ResponseFormat.MARKDOWN, context_extraction_mode=ContextExtractionMode.QUERY_ENGINE
# )

In [ ]:
from echo.step_templates.discovery import aget_research_data_for_client
from echo.utils import get_llm
import nest_asyncio
import asyncio

nest_asyncio.apply()



data = asyncio.run(aget_research_data_for_client(inputs, llm=get_llm()))